<a href="https://colab.research.google.com/github/seirah-yang/BootCamp/blob/main/langgraph_RAG_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colab-xterm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 11.7 MB/s eta 0:00:00


In [2]:
%load_ext colabxterm

In [4]:
!pip install langchain langgraph langchain-community chromadb sqlite-utils


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.4 MB/s eta 0:00

In [6]:
from langchain_community.llms import Ollama  # Ollama LLM 사용
from langchain_core.prompts import PromptTemplate  # 프롬프트 템플릿
from langgraph.graph import StateGraph, END  # LangGraph 상태 머신
from typing import TypedDict  # 타입 정의용
import sqlite3  # SQLite 사용
from langchain_community.document_loaders import TextLoader  # 텍스트 문서 로더
from langchain.text_splitter import CharacterTextSplitter  # 문서 분할 도구
from langchain_community.vectorstores import Chroma  # Chroma 벡터스토어
from langchain.embeddings import OllamaEmbeddings  # Ollama 임베딩 생성
from langchain.chains.combine_documents import create_stuff_documents_chain  # 문서 기반 QA 체인

# 0. RAG 벡터스토어 생성 (최초 1회 실행 후 주석처리 가능) = RAG하기위한 준비
def build_rag_chroma():
    with open("rag_data/medical.txt", "w", encoding="utf-8") as f:  # 텍스트 파일 저장
        f.write("""감기는 콧물, 기침, 인후통 등의 가벼운 증상이 있는 바이러스성 감염병입니다.\n
                    독감은 인플루엔자 바이러스 감염으로 고열, 근육통, 기침, 피로가 발생합니다.\n
                    코로나19는 고열, 마른기침, 후각상실 등을 유발하며 중증으로 발전할 수 있습니다.""")

    loader = TextLoader("rag_data/medical.txt", encoding="utf-8")  # 텍스트 로드
    docs = loader.load()  # 문서 리스트 로드
    splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=20)  # 텍스트 분할기
    split_docs = splitter.split_documents(docs)  # 문서 나누기

    embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")  # 임베딩 생성기
    db = Chroma.from_documents(split_docs, embedding=embeddings, persist_directory="./rag_chroma")  # Chroma 저장(벡터DB)
    db.persist()  # 저장

    build_rag_chroma()  # 최초 실행 시만 주석 해제

In [8]:
# 1. SQLite 샘플 데이터베이스 생성 (최초 1회 실행 후 주석처리 가능)
def build_sqlite_db():
    conn = sqlite3.connect("disease_knowledge.db")  # DB 연결
    cursor = conn.cursor()  # 커서 생성
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS diseases (
            name TEXT PRIMARY KEY,
            care TEXT
        )
    """)  # 테이블 생성
    sample_data = [
        ("감기", "수분을 충분히 섭취하고 휴식을 취하세요. 필요 시 해열제를 복용하세요."),
        ("독감", "타미플루 등 항바이러스제를 초기 투여하고 휴식이 필요합니다."),
        ("코로나19", "자가격리와 함께 해열제 복용 및 호흡기 증상 관리가 필요합니다."),
    ]
    cursor.executemany("INSERT OR IGNORE INTO diseases VALUES (?, ?)", sample_data)  # 데이터 삽입
    conn.commit()  # 커밋
    conn.close()  # 연결 종료

build_sqlite_db()  # 최초 실행 시만 주석 해제

In [9]:
!pip install grandalf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.2 MB/s eta 0:00:00


In [10]:
# 2. 상태 정의
class AgentState(TypedDict):  # 상태 타입 정의
    query: str  # 사용자 질의
    symptoms: str  # 추출된 증상
    disease_candidates: str  # 질병 후보
    disease_info: str  # 질병 설명 정보 (RAG)
    result: str  # 최종 응답

# 3. LLM 초기화
llm = Ollama(model="exaone3.5:2.4b")  # Ollama 모델 로딩

# 4. 에이전트 정의
extractor_prompt = PromptTemplate.from_template("""
사용자의 질문에서 증상에 해당하는 단어 또는 구를 추출하세요.
결과는 쉼표로 구분된 문자열로 출력하세요.
질문: {query}
""")  # 증상 추출 프롬프트

def extractor_agent(state: AgentState):  # 증상 추출 함수
    chain = extractor_prompt | llm  # 프롬프트 체인
    symptoms = chain.invoke({"query": state["query"]})  # LLM 실행
    return {**state, "symptoms": symptoms.strip()}  # 상태에 추가

matcher_prompt = PromptTemplate.from_template("""
다음 증상 목록을 바탕으로 가장 가능성 높은 질병 이름 3개를 쉼표로 추정하세요.
증상: {symptoms}
""")  # 질병 후보 추정 프롬프트

def matcher_agent(state: AgentState):  # 질병 후보 추정
    chain = matcher_prompt | llm
    candidates = chain.invoke({"symptoms": state["symptoms"]})
    return {**state, "disease_candidates": candidates.strip()}

def rag_agent(state: AgentState):  # RAG 검색 기반 질병 설명 생성
    embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
    vectorstore = Chroma(persist_directory="./rag_chroma", embedding_function=embeddings)
    retriever = vectorstore.as_retriever()

    rag_prompt = PromptTemplate.from_template("""
                                                사용자의 증상: {symptoms}

                                                다음 참고 문서를 바탕으로 관련 질병 정보를 요약해 주세요:
                                                {context}
                                                """)
    rag_chain = create_stuff_documents_chain(llm, rag_prompt)
    docs = retriever.get_relevant_documents(state["symptoms"])
    rag_response = rag_chain.invoke({"symptoms": state["symptoms"], "context": docs})
    return {**state, "disease_info": rag_response.strip()}

def sql_agent(state: AgentState):  # DB에서 질병 대처방안 조회
    conn = sqlite3.connect("disease_knowledge.db")
    cursor = conn.cursor()
    names = [n.strip() for n in state["disease_candidates"].split(",")]
    results = []
    for name in names:
        cursor.execute("SELECT care FROM diseases WHERE name = ?", (name,))
        row = cursor.fetchone()
        if row:
            results.append(f"{name} 대처법: {row[0]}")
        else:
            results.append(f"{name}: 대처 정보 없음")
    conn.close()
    return {**state, "disease_info": state["disease_info"] + "\n" + "\n".join(results)}
                                                                                    # 질병정보와 SQL 결과(care)를 문자열로 합친 것
                                                                                            # 예시: 감기는 따뜻한 물을 마시고, 충분한 숙면을 취하세요 .
                                                                                            # ---> 관련 정보 및 대처방안 :{dz.info} RAG + SQL 결과의 합 로 입력
answer_prompt = PromptTemplate.from_template("""
사용자의 증상은 다음과 같습니다: {symptoms}

예측된 질병 후보: {disease_candidates}

관련 정보 및 대처방안:
{disease_info}

위 내용을 바탕으로 사용자에게 알기 쉽게 설명해주세요.
""")  # 최종 응답 생성 프롬프트

def answer_agent(state: AgentState):  # 응답 생성 에이전트
    chain = answer_prompt | llm  # 프롬프트와 LLM을 연결하여 실행 체인 구성
    answer = chain.invoke({
        "symptoms": state["symptoms"],
        "disease_candidates": state["disease_candidates"],
        "disease_info": state["disease_info"]
    })
    return {**state, "result": answer.strip()}

# 5. LangGraph 정의
from langgraph.graph import StateGraph  # LangGraph 구성 요소

graph = StateGraph(AgentState)  # 그래프 정의
graph.add_node("extractor", extractor_agent)  # 노드 추가
graph.add_node("matcher", matcher_agent)
graph.add_node("rag", rag_agent)
graph.add_node("sql_lookup", sql_agent)
graph.add_node("answer", answer_agent)

graph.set_entry_point("extractor")  # 시작 노드 설정
graph.add_edge("extractor", "matcher")  # 노드 간 연결 정의
graph.add_edge("matcher", "rag")
graph.add_edge("rag", "sql_lookup")
graph.add_edge("sql_lookup", "answer")
graph.add_edge("answer", END)  # 종료 노드 설정

app = graph.compile()  # 그래프 컴파일

print("============================== LangGraph 구조:")
app.get_graph().print_ascii()  # 구조 출력

# 6. 실행 예시
query = "기침이 심하고 목이 아프고 열이 납니다"  # 사용자 질문
result = app.invoke({"query": query})  # 실행

print("============================== 최종 응답:")
print(result["result"])  # 결과 출력

============================== LangGraph 구조:
+-----------+  
| __start__ |  
+-----------+  
       *       
       *       
       *       
+-----------+  
| extractor |  
+-----------+  
       *       
       *       
       *       
  +---------+  
  | matcher |  
  +---------+  
       *       
       *       
       *       
    +-----+    
    | rag |    
    +-----+    
       *       
       *       
       *       
+------------+ 
| sql_lookup | 
+------------+ 
       *       
       *       
       *       
  +--------+   
  | answer |   
  +--------+   
       *       
       *       
       *       
  +---------+  
  | __end__ |  
  +---------+  


/tmp/ipython-input-2029814874.py:35: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="exaone3.5:2.4b")
/tmp/ipython-input-2029814874.py:36: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory="./rag_chroma", embedding_function=embeddings)
/tmp/ipython-input-2029814874.py:46: LangChainDeprecationWarning: The method `BaseRe

============================== 최종 응답:
당신의 증상—기침, 목 아픔, 그리고 발열—은 몇 가지 주요 호흡기 감염 질환과 연관성이 높아 보입니다. 주요 질병들을 간단하게 요약해드리겠습니다:

### 주요 질병 요약

1. **인후염 (목 감염)**
   - **특징**: 주로 목 아픔과 기침이 특징적입니다. 때로는 편도선에 염증이 생길 수 있어 삼키는 것이 어려워질 수 있습니다.
   - **원인**: 바이러스(예: 아데노바이러스) 또는 박테리아(예: 스테포코커스 폐렴균)에 의해 발생할 수 있습니다.
   - **대처 방법**:
     - 충분한 휴식과 수분 섭취로 몸 상태 유지
     - 목을 부드럽게 마사지하거나 따뜻한 수건으로 찜질하기
     - 기침과 목 아픔을 완화시키기 위해 적절한 항염증제나 가습기 사용 고려

2. **감기 (Common Cold)**
   - **특징**: 기침, 목 아픔, 발열 등이 나타날 수 있습니다. 초기에는 건조한 기침부터 시작되며, 발열은 일부 환자에게만 나타납니다.
   - **원인**: 주로 바이러스에 의해 발생합니다 (예: RSV, 코로나바이러스).
   - **대처 방법**:
     - 충분한 수분 섭취로 탈수 예방
     - 가습기를 사용해 공기 습도 유지
     - 필요 시 기침 억제제 사용
     - 충분한 휴식과 휴식이 중요합니다

3. **독감 (Influenza)**
   - **특징**: 감기와 유사하지만 보통 더 심각한 발열과 전신 증상이 동반됩니다. 기침과 목 아픔도 나타날 수 있습니다.
   - **원인**: 인플루엔자 바이러스에 의해 발생합니다.
   - **대처 방법**:
     - 충분한 휴식과 영양 섭취
     - 열이 심할 경우 해열제 복용 (의사의 지시에 따라)
     - 감염 예방을 위해 손 씻기와 사회적 거리두기 실천

### 추천 사항
- **진단**: 정확한 진단을 위해서는 반드시 의료 전문가와 상담해야 합니다. 병력 청취와 필요한 경